In [1]:
import warnings
warnings.filterwarnings("ignore")
%load_ext autoreload
%autoreload 2
from qiskit_metal import Dict, MetalGUI, designs

from qiskit_metal.qlibrary.qubits.transmon_pocket import TransmonPocket
from qiskit_metal.qlibrary.qubits.transmon_pocket_6 import TransmonPocket6
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
from qiskit_metal.qlibrary.tlines.straight_path import RouteStraight
from qiskit_metal.qlibrary.terminations.launchpad_wb_coupled import LaunchpadWirebondCoupled
from qiskit_metal.qlibrary.terminations.short_to_ground import ShortToGround
from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee
from qiskit_metal.qlibrary.user_components.left_coupler import LeftCoupler
from qiskit_metal.qlibrary.user_components.right_coupler import RightCoupler
from qiskit_metal.qlibrary.user_components.straightdown_coupler import StraightDownCoupler
from qiskit_metal.qlibrary.user_components.straightup_coupler import StraightUpCoupler

In [2]:
design = designs.DesignFlipChip({}, overwrite_enabled=True)
gui = MetalGUI(design)

In [3]:
design.chips['C_chip'].size['size_x']='3mm'
design.chips['C_chip'].size['size_y']='3mm'
design.chips['Q_chip'].size['size_x']='2mm'
design.chips['Q_chip'].size['size_y']='2mm'
design.chips['Q_chip'].size['center_z']='6um'

In [4]:
design.get_chip_z('Q_chip')

'6um'

In [5]:

cp0 = dict(down = dict(loc_W=1, loc_H=-1, pad_width='120um'))
q0=[TransmonPocket6(design, 'Q0', options=dict(pos_x='-300um',
                                            pos_y='-500um',
                                            chip='Q_chip',
                                            pad_width='370um',
                                            pad_height='100um',
                                            pad_gap='80um',
                                            pocket_width='680um',
                                            pocket_height='680um'))]
                                            
qubits=q0
q0x=float(qubits[0].options.pos_x.strip('um'))
q0y=float(qubits[0].options.pos_y.strip('um'))


c0 = StraightUpCoupler(design, "c0", options=dict(pos_x=f"{q0x-120}um",
                                              pos_y=f"{q0y+90}um",
                                              pad_width="165um",
                                              pad_height="100um",
                                              cpw_stub_height="200um",
                                              chip="C_chip"))
design.rebuild()
gui.rebuild()
gui.autoscale()

In [6]:
lpL=LaunchpadWirebondCoupled(design, 'lpL',
                                         options = dict(chip="C_chip",
                                         pos_x = '-700um',
                                         pos_y = '800um',
                                         lead_length = '30um'))
lpR=LaunchpadWirebondCoupled(design, 'lpR',
                                         options = dict(chip="C_chip",
                                         pos_x = '700um',
                                         pos_y = '800um',
                                         orientation='180',
                                         lead_length = '30um'))                                        

In [7]:
stg=ShortToGround(design, 'stg', options=Dict(chip="C_chip",
                                                pos_x='0.65mm',
                                                pos_y='-0.2mm',
                                                orientation='270'))

In [8]:
fl0=RouteStraight(design, 'fl0',options=Dict(chip="C_chip", pin_inputs=Dict(start_pin=Dict(
                                                            component='lpL',
                                                            pin='tie'),
                                                            end_pin=Dict(
                                                            component='lpR',
                                                            pin='tie'))))


# Readout resonator

In [9]:
from collections import OrderedDict
jogs=OrderedDict()
def connect(component_name: str, component1: str, pin1: str, component2: str, pin2: str,
            length: str, start_jogged_extension: OrderedDict, end_jogged_extension: OrderedDict,
            spacing: str, asymmetry='0 um',  start_straight='0 um', end_straight='0 um', flip=False):
    """Connect two pins with a CPW."""
    myoptions = Dict(
        chip="C_chip",
        pin_inputs=Dict(
            start_pin=Dict(
                component=component1,
                pin=pin1),
            end_pin=Dict(
                component=component2,
                pin=pin2)),
        lead=Dict(
            start_straight=start_straight,
            end_straight=end_straight,
            start_jogged_extension=start_jogged_extension,
            end_jogged_extension=end_jogged_extension),
        total_length=length,
        fillet = '99.9um',
        meander=dict(asymmetry=asymmetry,
                     spacing=spacing))
    myoptions.meander.lead_direction_inverted = 'true' if flip else 'false'
    return RouteMeander(design, component_name, myoptions)

# Resonator length
3.85mm

In [10]:
from qiskit_metal.analyses.em.cpw_calculations import guided_wavelength

def find_resonator_length(frequency, line_width, line_gap, N): 
    [lambdaG, etfSqrt, q] = guided_wavelength(frequency*10**9, line_width*10**-6,
                                              line_gap*10**-6, 750*10**-6, 200*10**-9)
    return str(lambdaG/N*10**3)+" mm"

In [11]:
resonator_length_4=[]
a=find_resonator_length(7.5,10,6,4)
a=a.strip('mm')
a=float(a)-0.25 +0.5-0.23-0.5# subtract coupler length, for +0.5Ghz, coupler length
resonator_length_4.append(a)
print(resonator_length_4)


[3.5775622473488387]


In [12]:
jogs1=OrderedDict()
jogs1[0]=['R','300um']
jogs1[1]=['R', '300um']
jogs1[2]=['L','700um']
jogs1[3]=['R','200um']
jogs1[4]=['R','670um']
jogs1[5]=['L','200um']
jogs1[6]=['L','740um']


In [13]:
rr0=connect('rr0', 'c0', 'in','stg', 'short',
            resonator_length_4[0],jogs1,jogs, 
            '100um','0um', '944um', '100um')
            

# EPR Analysis

In [14]:
from qiskit_metal.analyses.quantization import EPRanalysis
eig_rd = EPRanalysis(design, "hfss")
hfss = eig_rd.sim.renderer

In [15]:
hfss.start()

INFO 05:39PM [connect_project]: Connecting to Ansys Desktop API...
INFO 05:39PM [load_ansys_project]: 	Opened Ansys App
INFO 05:39PM [load_ansys_project]: 	Opened Ansys Desktop v2018.0.0
INFO 05:39PM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/kimje/Documents/Ansoft/
	Project:   Flipchip Bump
INFO 05:39PM [connect_design]: 	Opened active design
	Design:    Design_q3d [Solution type: Q3D]
INFO 05:39PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.AnsysQ3DSetup'>)
INFO 05:39PM [connect]: 	Connected to project "Flipchip Bump" and design "Design_q3d" 😀 



True

In [16]:
hfss.activate_ansys_design("flip6um_V7", 'eigenmode')
hfss.render_design(['Q0','c0','lpL','lpR','stg','rr0','fl0'],[],
                    box_plus_buffer=False)

05:39PM 04s WARNING [activate_ansys_design]: The design_name=flip6um_V7 was not in active project.  Designs in active project are: 
['flip7um_BUMP', 'flip7um_V6', 'flip8um_V6_BUMP', 'flip6um_V6', 'flip7um_V7', 'flip6um_BUMP', 'flip8um_V6', 'Design_q3d'].  A new design will be added to the project.  
INFO 05:39PM [connect_design]: 	Opened active design
	Design:    flip6um_V7 [Solution type: Eigenmode]
WARNING 05:39PM [connect_setup]: 	No design setup detected.
WARNING 05:39PM [connect_setup]: 	Creating eigenmode default setup.
INFO 05:39PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)


In [17]:
hfss.close()


True

# LOM Analysis

In [18]:
from qiskit_metal.analyses.quantization import LOManalysis
c1 = LOManalysis(design, "q3d")

In [19]:
c1.sim.setup.max_passes = 10
c1.sim.setup_update(freq_ghz='5.66',solution_order = 'Medium', auto_increase_solution_order = 'False')
c1.sim.setup

{'name': 'Setup',
 'reuse_selected_design': True,
 'reuse_setup': True,
 'freq_ghz': '5.66',
 'save_fields': False,
 'enabled': True,
 'max_passes': 10,
 'min_passes': 2,
 'min_converged_passes': 2,
 'percent_error': 0.5,
 'percent_refinement': 30,
 'auto_increase_solution_order': 'False',
 'solution_order': 'Medium',
 'solver_type': 'Iterative'}

In [21]:
#c1.sim.run(components=['Q0','c0'], open_terminations=[('c0', 'in')])
c1.sim._get_results_from_renderer()
c1.sim.capacitance_matrix

INFO 05:56PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmp5b0xxwxh.txt, C, , Setup:LastAdaptive, "Original", "ohm", "nH", "fF", "mSie", 5660000000, Maxwell, 1, False
INFO 05:56PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpbnxaxsxq.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5660000000, Maxwell, 1, False
INFO 05:56PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmplti73hex.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5660000000, Maxwell, 2, False
INFO 05:56PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpe1_nh936.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5660000000, Maxwell, 3, False
INFO 05:56PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpx414k39d.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5660000000, Maxwell, 4, False
INFO 05:56

,pad_bot_Q0,pad_top_Q0,quad_coupler_c0,SignalNet1
pad_bot_Q0,140.0646,-11.4703,-0.09060,-127.12160
pad_top_Q0,-11.4703,140.0387,-26.09150,-100.94600
quad_coupler_c0,-0.0906,-26.0915,118.59440,-92.12913
SignalNet1,-127.1216,-100.9460,-92.12913,384.69170


In [24]:
c1.setup.junctions = Dict({'Lj': 12, 'Cj': 2})
c1.setup.freq_readout = 7.18
c1.setup.freq_bus = []
c1.run_lom()
c1.lumped_oscillator_all

[1, 2] [3]
Predicted Values

Transmon Properties
f_Q 4.917777 [GHz]
EC 245.960551 [MHz]
EJ 13.616300 [GHz]
alpha -280.084042 [MHz]
dispersion 3.901703 [KHz]
Lq 11.995161 [nH]
Cq 78.753394 [fF]
T1 369.904507 [us]

**Coupling Properties**

tCqbus1 -5.119116 [fF]
gbus1_in_MHz -68.115388 [MHz]
χ_bus1 -0.470082 [MHz]
1/T1bus1 430.259540 [Hz]
T1bus1 369.904507 [us]
Bus-Bus Couplings


,fQ,EC,EJ,alpha,dispersion,gbus,chi_in_MHz,χr MHz,gr MHz
1,6.19414,402.970306,13.6163,-482.18249,327.108493,[-54.84533288007124],[-2.021143606960377],2.021144,54.845333
2,4.974403,252.00173,13.6163,-287.532439,4.98775,[-55.60706333135122],[-0.33570298065455173],0.335703,55.607063
3,4.965135,251.007274,13.6163,-286.304584,4.793113,[-65.51813613718048],[-0.4607677318985576],0.460768,65.518136
4,4.951234,249.519932,13.6163,-284.469453,4.514058,[-64.90624125603226],[-0.44457624292859943],0.444576,64.906241
5,4.937074,248.009942,13.6163,-282.607957,4.244992,[-66.05143274601805],[-0.4525181392450076],0.452518,66.051433
6,4.932433,247.516263,13.6163,-281.999702,4.160036,[-67.59543956781812],[-0.47125153158513877],0.471252,67.595440
7,4.920838,246.28503,13.6163,-280.483456,3.954427,[-67.97036476611085],[-0.46982500380330966],0.469825,67.970365
8,4.917777,245.960551,13.6163,-280.084042,3.901703,[-68.11538804213545],[-0.47008216052288165],0.470082,68.115388


In [23]:
c1.sim.close()

Warning! 3 COM references still alive
Ansys will likely refuse to shut down
